# records

> Module fore records

In [ ]:
#| default_exp records

In [ ]:
#| hide 
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from dataclasses import dataclass
from typing import List

import numpy as np
import numpy.typing as npt
from typing import Dict, List, Optional, Tuple, Union
from enum import Enum, auto
from typing import ClassVar, List

In [ ]:
#| export
@dataclass
class TimeseriesRecording:
    """Base class for data that is oriented along an array representing time.

    Subclasses of this add specific data types, such as PSG labels or triaxial accelerometer readings.
    """

    time: npt.NDArray[np.float64]  # seconds

    def __post_init__(self):
        self.time = np.squeeze(self.time)
        assert len(self.time.shape) == 1

    def bin_based_on(self, time: np.ndarray):
        raise NotImplementedError

    def align_self_and_other_series(self, other: "TimeseriesRecording") -> None:
        """
        MUTATING METHOD. Changes both `self` and `other` so that only the shared times are kept.

        That is, when this method completes, both self.time and other.time contain only timestamps
        common to the two objects before this method was called.
        """
        start = max(self.time.min(), other.time.min())
        end = min(self.time.max(), other.time.max())

        self.trim(start, end)
        other.trim(start, end)

    def fill_gaps(self, max_gap_seconds: float, fill_dt: float):
        gaps = self._identify_gaps(max_gap_seconds=max_gap_seconds)

        if len(gaps) == 0:
            return

        fills = [np.arange(self.time[i], self.time[i + 1], fill_dt) for i in gaps]

        self.time = fill_numpy(base_array=self.time, fills=fills, start_idx=gaps)

        self._fill_gaps_specific_data(gaps, fills)

    def sort_by_time(self) -> None:
        sort_idx = np.argsort(self.time)
        self._sort_specific_data(sort_idx)
        self.time = self.time[sort_idx]

    def _identify_gaps(self, max_gap_seconds: float) -> npt.NDArray[np.int64]:
        """Finds places where the time step between self.time indices exceeds the given maximum.

        Parameters
        ---
        `max_gap_seconds`: `float`
            If `np.diff(self.time)[i] > max_gap_seconds`, then `self.time[i]` is reported as a "gap" by including `i` in the returned array.

        Returns
        ---
        Indices `i` of `self.time` such that `self.time[i+1] - self.time[i] > max_gap_seconds`.
        """
        return np.flatnonzero(np.diff(self.time) > max_gap_seconds)

    def _fill_gaps_specific_data(
        self, gaps: npt.NDArray[np.int64], time_fillers: List[np.ndarray]
    ):
        """
        Used by subclasses to fill their specific data. Arguments show indices where fillers were inserted and the time arrays that were inserted.

        Parameters
        ---
        gaps: NumPy array of integers
            Typically the output of self._identify_gaps. Indicates where in the time axis values were inserted (start indices).
        time_fillers: List[np.ndarray]
            The time arrays that were added. Subclasses might use this only to compute the length of the fillers to add, or might insert values that are a function of the times.
        """
        return

    def _sort_specific_data(self, sort_idx: npt.NDArray[np.int64]) -> None:
        """
        Used in subclasses of TimeseriesRecording to sort type-specific data.

        ** Mutating function **

        Parameters:
        ---
        sort_idx: NumPy array of ints, usually the output of np.argsort on self.time
            Subclasses should implement a version of this that handles re-organizing their specific data in the order given
        """
        return

    def _trim_specific_data(self, select_idx: np.ndarray) -> None:
        """
        Used in subclasses of TimeseriesRecording to trim type-specific data based on an array of indices to include.

        ** Mutating function **

        Parameters:
        ---
        select_idx: NumPy array of ints or bools, usually the output of self._trim_time
            Subclasses should implement a version of this that handles subsetting their specific data as specified using this array.
            This method should support select_idx that provides indices to include OR booleans, one per element, indicating if should include.
            E.g.: if our data is `np.array([1, 2, 3, 4])` then it should be equivalent to call this function with
            `select_idx = np.array([1, 3])` or `select_idx = np.array([False, True, False, True])`
        """
        return

    def trim(self, start: float, end: float) -> None:
        select_idx = self._trim_time(start, end)
        self._trim_specific_data(select_idx)

    def _trim_time(self, start: float, end: float) -> np.ndarray:
        select = (start <= self.time) & (self.time <= end)
        self.time = self.time[select]

        return select

@dataclass
class AccelerometerRecord(TimeseriesRecording):
    _xyz: np.ndarray
    Hz: int | float | None = None

    def __post_init__(self):
        super().__post_init__()
        assert self._xyz.shape == self._expected_xyz_shape

    def bin_based_on(self, time: np.ndarray):
        return

    @property
    def _expected_xyz_shape(self) -> Tuple[int, int]:
        return (len(self.time), 3)

    def _fill_gaps_specific_data(
        self, gaps: npt.NDArray[np.int64], time_fillers: List[np.ndarray]
    ):
        zero_fills = [np.zeros((len(f), 3)) for f in time_fillers]
        self._xyz = fill_numpy(self._xyz, fills=zero_fills, start_idx=gaps)

    def resample(self, Hz: float):
        self.sort_by_time()
        new_time = np.arange(self.time[0], self.time[-1], 1 / Hz)

        new_xyz = np.zeros((len(new_time), 3))

        for j in range(3):
            new_xyz[:, j] = np.interp(new_time, self.time, self.xyz[:, j])

        self.time = new_time
        self.xyz = new_xyz

    @property
    def x(self) -> np.ndarray:
        return self._xyz[:, 0]

    @x.setter
    def x(self, value):
        self._xyz[:, 0] = value

    @property
    def y(self) -> np.ndarray:
        return self._xyz[:, 1]

    @y.setter
    def y(self, value):
        self._xyz[:, 1] = value

    @property
    def z(self) -> np.ndarray:
        return self._xyz[:, 2]

    @z.setter
    def z(self, value):
        self._xyz[:, 2] = value

    def trim(self, start: float, end: float) -> None:
        select = self._trim_time(start, end)
        self._xyz = self._xyz[select]

    @property
    def l2_norm(self) -> np.ndarray:
        return np.sqrt(self.x * self.x + self.y * self.y + self.z * self.z)

    @property
    def xyz(self) -> np.ndarray:
        return self._xyz

    @xyz.setter
    def xyz(self, value):
        assert value.shape == self._expected_xyz_shape
        self._xyz = value

    @property
    def txyz(self) -> np.ndarray:
        stacked = np.vstack((self.time.T, self._xyz))
        # stacked = np.vstack((self.time, self.x, self.y, self.z))
        # stacked = stacked.T # shape (N, 4) instead of (4, N); N = len(self.x)
        return stacked

@dataclass
class ActivityCountRecord(TimeseriesRecording):
    count: np.ndarray

    def _fill_gaps_specific_data(
        self, gaps: npt.NDArray[np.int64], time_fillers: List[np.ndarray]
    ):
        self.count = fill_numpy(
            self.count,
            fills=[np.zeros_like(t) for t in time_fillers],  # fill with zeros like time
            start_idx=gaps,
        )

    def bin_based_on(self, time: np.ndarray):
        bins = compute_bins(self.time, time)
        valid_bins = bins >= 0  # sometimes trim didn't work so well??
        bins = bins[valid_bins]
        weights = self.count[valid_bins]
        binned = np.bincount(bins, weights=weights)
        if len(time) != len(binned):
            length = min(len(time), len(binned))
            time = time[:length]
            binned = binned[:length]

        self.time = time
        self.count = binned

    def trim(self, start: float, end: float) -> None:
        select = self._trim_time(start, end)
        self.count = self.count[select]

        
class SleepOrWake(Enum):
    """
    Very simple enum for fixing, once-and-for-all, meaning of class 0 and 1.
    Use this enum when sensible for clarity in code.
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    SLEEP = 1

    
class SleepStagesWLDR(Enum):
    """
    Similar to SleepStages(Enum), but for Wake/Light/Deep/REM instead of Ns
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    LIGHT = 1
    DEEP = 2
    REM = 3

    def to_masked_sleep_wake(self) -> SleepOrWake:
        match self:
            case SleepStagesWLDR.DO_NOT_USE:
                return SleepOrWake.DO_NOT_USE
            case SleepStagesWLDR.UNSCORED:
                return SleepOrWake.UNSCORED
            case SleepStagesWLDR.WAKE:
                return SleepOrWake.WAKE
            case _:
                return SleepOrWake.SLEEP
    
class SleepStages(Enum):
    """
    Very simple enum for fixing, once-and-for-all, meaning of integer sleep stages.
    Use this enum when possible for clarity in code.
    """

    DO_NOT_USE = -2  # excluded post-hoc, e.g. padding
    UNSCORED = -1
    WAKE = 0
    N1 = 1
    N2 = 2
    N3 = 3
    N4 = 4
    REM = 5

    def to_masked_sleep_wake(self) -> SleepOrWake:
        return self.to_WLDR().to_masked_sleep_wake()

    def to_WLDR(self) -> SleepStagesWLDR:
        match self:
            case SleepStages.DO_NOT_USE:
                return SleepOrWake.DO_NOT_USE
            case SleepStages.UNSCORED:
                return SleepStagesWLDR.UNSCORED
            case SleepStages.WAKE:
                return SleepStagesWLDR.WAKE
            case SleepStages.N1 | SleepStages.N2:
                return SleepStagesWLDR.LIGHT
            case SleepStages.N3 | SleepStages.N4:
                return SleepStagesWLDR.DEEP
            case SleepStages.REM:
                return SleepStagesWLDR.REM




PSG_Enums = SleepOrWake | SleepStagesWLDR | SleepStages


class SleepClassificationProblem(Enum):
    SLEEP_OR_WAKE = auto()
    SLEEP_STAGES_WLDR = auto()
    SLEEP_STAGES = auto()

    def type_enum(self) -> PSG_Enums:
        match self:
            case SleepClassificationProblem.SLEEP_OR_WAKE:
                return SleepOrWake
            case SleepClassificationProblem.SLEEP_STAGES_WLDR:
                return SleepStagesWLDR
            case SleepClassificationProblem.SLEEP_STAGES:
                return SleepStages


@dataclass
class PSGRecord(TimeseriesRecording):
    psg: List[PSG_Enums]
    label_type: ClassVar[PSG_Enums]

    def __post_init__(self):
        super().__post_init__()
        assert len(self.psg) == len(self.time)

    def _sort_specific_data(self, sort_idx: npt.NDArray[np.int64]) -> None:
        self.psg = [self.psg[j] for j in sort_idx]

    def _trim_specific_data(self, select_idx: np.ndarray) -> None:
        if select_idx.dtype == bool:
            # Reduce to case where indices to include are provided
            select_idx = np.nonzero(select_idx)[0]
        self.psg = [self.psg[j] for j in select_idx]

    def _fill_gaps_specific_data(
        self, gaps: npt.NDArray[np.int64], time_fillers: List[np.ndarray]
    ):
        for gap_idx, filler in zip(gaps, time_fillers):
            fill_array = [self.label_type.UNSCORED] * len(filler)
            self.psg = inserting_at_index(self.psg, fill_array, gap_idx + 1)

    def bin_based_on(self, time: np.ndarray):
        pass

    def to_sleep_wake(
        self,
        unscored_as_wake: bool = True,
    ) -> "PSGSleepWakeRecord":
        raise NotImplementedError

    def labels(
        self,
        unscored_as_wake: bool = True,
    ) -> np.ndarray:
        # Make a transformation that either sends -1 to 0, or leaves as is.
        def xfrm(x: int) -> int:
            # xfrm = (lambda x: max(x, 0)) if unscored_as_wake else (lambda x: x)
            if x == SleepOrWake.DO_NOT_USE.value:
                return x
            return max(x, 0) if unscored_as_wake else x

        return np.array([xfrm(elt.value) for elt in self.psg], dtype=np.int64)

    def sw_labels(
        self,
        unscored_as_wake: bool = True,
    ) -> np.ndarray:
        return self.to_sleep_wake().labels(unscored_as_wake=unscored_as_wake)

    def __len__(self):
        return len(self.psg)


class PSGPredictionsShapeError(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)


class PSGPredictionsNotProbabilityVectorError(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)


@dataclass
class PSGModelOutputs(TimeseriesRecording):
    """
    TimeseriesRecording output from models; models should indicate in their
    implementation of evaluation which subclass of this is returned.

    Probabilities should be a NumPy array shaped such that
        self.probabilities[i]
    indicates the probabilities of each sleep stage for times between
        self.time[i] and self.time[i+1]
    In particular, self.probabilities should have shape
        (N,) or (N, m)
    where N == len(self.time) and m = number of stages
    """

    probabilities: np.ndarray

    def __post_init__(self):
        try:
            assert len(self.time) == len(self.probabilities)
        except AssertionError:
            raise PSGPredictionsShapeError("len(probabilities) must equal len(time)")

        try:
            # Do not require >=0, allows for masking
            assert np.all(self.probabilities <= 1.0)
        except AssertionError:
            raise PSGPredictionsNotProbabilityVectorError(
                "Probabilities vector must contain entries between 0.0 and 1.0, inclusive."
            )

    def _sort_specific_data(self, sort_idx: npt.NDArray[np.int64]) -> None:
        self.probabilities = self.probabilities[sort_idx]

    def _trim_specific_data(self, select_idx: np.ndarray) -> None:
        if select_idx.dtype == bool:
            # Reduce to case where indices to include are provided
            select_idx = np.nonzero(select_idx)
        self.probabilities = self.probabilities[select_idx]


class PSGSleepWakePredictions(PSGModelOutputs):
    def __post_init__(self):
        super().__post_init__()
        try:
            assert self.probabilities.shape == (len(self.time),)
        except AssertionError:
            raise PSGPredictionsShapeError("len(probabilities) must equal len(time)")


class PSGStagingPredictions(PSGModelOutputs):
    n_stages: int

    def __post_init__(self):
        super().__post_init__()

        try:
            assert np.all(
                (np.sum(self.probabilities, axis=-1) == 1.0)
                | (np.min(self.probabilities, axis=-1) < 0.0)
            )
        except AssertionError:
            problem_idx = np.nonzero(np.sum(self.probabilities, axis=-1) != 1.0)
            return PSGPredictionsNotProbabilityVectorError(
                "Did not get a vector whose axis-0 arrays sum to 1.0\n"
                + f"Problems at indices:\n{problem_idx}"
            )

    def to_sleep_wake_predictions(self) -> PSGSleepWakePredictions:
        return PSGSleepWakePredictions(
            time=self.time,
            probabilities=(1 - self.probabilities[:, 0]),  # P(sleep) = 1 - P(wake)
        )

    

@dataclass
class PSGSleepWakeRecord(PSGRecord):
    label_type = SleepOrWake
    psg: List[SleepOrWake]

    def to_sleep_wake(self) -> "PSGSleepWakeRecord":
        return self

    @staticmethod
    def from_predictions(
        predictions: PSGSleepWakePredictions, threshold: float = 0.5
    ) -> "PSGSleepWakeRecord":
        """
        Converts probabilistic outputs to definitive sleep-wake.
        """

        def probas_to_class(p: float) -> SleepOrWake:
            if p >= threshold:
                return SleepOrWake.SLEEP
            elif p >= 0:
                return SleepOrWake.WAKE
            elif p >= -1:
                return SleepOrWake.UNSCORED
            elif p < -1:
                return SleepOrWake.DO_NOT_USE

        binary_stages = [probas_to_class(p) for p in predictions.probabilities]

        return PSGSleepWakeRecord(time=predictions.time, psg=binary_stages)

    @classmethod
    def from_staging_predictions(
        cls, predictions: PSGStagingPredictions, threshold: float = 0.5
    ) -> "PSGSleepWakeRecord":
        return cls.from_predictions(
            predictions.to_sleep_wake_predictions(), threshold=threshold
        )

    def __len__(self):
        return len(self.psg)


@dataclass
class PSGStagingRecord(PSGRecord):
    label_type = SleepStages
    psg: List[SleepStages]

    def __post_init__(self):
        for j in range(len(self.psg)):
            if self.psg[j] == SleepStages.N4:
                self.psg[j] = SleepStages.N3

    def to_sleep_wake(self) -> PSGSleepWakeRecord:
        return PSGSleepWakeRecord(
            time=self.time, psg=[p.to_masked_sleep_wake() for p in self.psg]
        )

    def to_WLDR(self) -> "PSGStagingWLDRRecord":
        return PSGStagingWLDRRecord(
            time=self.time,
            psg=[p.to_WLDR() for p in self.psg],
        )

    def one_hot(self) -> np.ndarray:
        return one_hot_encoded(self.labels(), n_classes=len(self.label_type))


@dataclass
class PSGStagingWLDRRecord(PSGRecord):
    label_type = SleepStagesWLDR
    psg: List[SleepStagesWLDR]

    def to_sleep_wake(self) -> PSGSleepWakeRecord:
        return PSGSleepWakeRecord(
            time=self.time, psg=[p.to_masked_sleep_wake() for p in self.psg]
        )

    def to_WLDR(self) -> "PSGStagingWLDRRecord":
        return self

    def one_hot(self, unscored_as_wake: bool = False) -> np.ndarray:
        return one_hot_encoded(
            self.labels(unscored_as_wake=unscored_as_wake),
            n_classes=len(self.label_type),
        )


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
import pisces

In [ ]:
help(pisces)

Help on package pisces:

NAME
    pisces

PACKAGE CONTENTS
    _modidx
    coooooore
    core
    load
    records
    records2

VERSION
    0.0.1

FILE
    /Users/ojwalch/Documents/pisces/pisces/__init__.py


